# Project 3 - Part 1

## Business problem

- Produce a MySQL database on movies from a subset of IMDB's publicly available dataset.
- Use this database to analyze what makes a movie successful.
- Provide recommendations to the stakeholder on how to make a successful movie.

## Requirements

- Download several files from IMDB’s movie data set.
- Exclude any movie with missing values for genre or runtime.
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre).
- Include only movies that were released 2000 - 2021 (include 2000 and 2021).
- Include only movies that were released in the United States.

## Imports

In [1]:
import pandas as pd
import numpy as np
import os

## Load the data

In [2]:
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [4]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [5]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

## Inspect and filter the data: Title AKAs

**Data dictionary**

- titleId (string) - a tconst, an alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- title (string) – the localized title
- region (string) - the region for this version of the title
- language (string) - the language of the title
- types (array) - Enumerated set of attributes for this alternative title. E.g. "alternative", "dvd", "festival", "tv", etc.
- attributes (array) - Additional terms to describe this alternative title, not enumerated
- isOriginalTitle (boolean) – 0: not original title; 1: original title

In [6]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37544903 entries, 0 to 37544902
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


In [7]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [8]:
print(f'There are {akas.shape[0]} rows and {akas.shape[1]} columns in the title_akas dataset.')

There are 37544903 rows and 8 columns in the title_akas dataset.


In [9]:
## Align datatypes
akas['isOriginalTitle'] = akas['isOriginalTitle'].astype(bool)

In [10]:
## Replace "\N" with np.nan
akas.replace({'\\N':np.nan}, inplace=True)

In [11]:
## Keep only US movies
akas = akas[akas['region'] == 'US']

In [12]:
## Cleaned and filtered data
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472694 entries, 5 to 37544647
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1472694 non-null  object
 1   ordering         1472694 non-null  int64 
 2   title            1472694 non-null  object
 3   region           1472694 non-null  object
 4   language         4171 non-null     object
 5   types            984741 non-null   object
 6   attributes       47636 non-null    object
 7   isOriginalTitle  1472694 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 91.3+ MB


## Inspect and filter the data: Title Basics

**Data dictionary**

- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

In [13]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10253861 entries, 0 to 10253860
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 704.1+ MB


In [14]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [15]:
print(f'There are {basics.shape[0]} rows and {basics.shape[1]} columns in the title_basics dataset.')

There are 10253861 rows and 9 columns in the title_basics dataset.


In [16]:
## Align datatypes
basics['isAdult'] = basics['isAdult'].astype(bool)
basics['startYear'] = pd.to_datetime(basics['startYear'], format='%Y', errors='coerce')
basics['endYear'] = pd.to_datetime(basics['endYear'], format='%Y', errors='coerce')
basics['runtimeMinutes'] = pd.to_numeric(basics['runtimeMinutes'], errors='coerce')

In [17]:
## Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [18]:
## Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [19]:
## Eliminate movies that are null for genre
basics.dropna(subset=['genres'], inplace=True)

In [20]:
## Keep only titleType==Movie
basics = basics[basics['titleType'] == 'movie']

In [21]:
## Keep startYear 2000-2022
basics = basics[(basics['startYear'] >= '2000') & (basics['startYear'] < '2022')]

In [22]:
## Eliminate movies that include "Documentary" in genre
is_documentary = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_documentary]

In [23]:
## Keep only US movies
us_movies = basics['tconst'].isin(akas['titleId'])
basics = basics[us_movies]

In [24]:
## Cleaned and filtered data
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82146 entries, 34800 to 10253627
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   tconst          82146 non-null  object        
 1   titleType       82146 non-null  object        
 2   primaryTitle    82146 non-null  object        
 3   originalTitle   82146 non-null  object        
 4   isAdult         82146 non-null  bool          
 5   startYear       82146 non-null  datetime64[ns]
 6   endYear         0 non-null      datetime64[ns]
 7   runtimeMinutes  82146 non-null  float64       
 8   genres          82146 non-null  object        
dtypes: bool(1), datetime64[ns](2), float64(1), object(5)
memory usage: 5.7+ MB


## Inspect and filter the data: Title Ratings

**Data dictionary**

- tconst (string) - alphanumeric unique identifier of the title
- averageRating – weighted average of all the individual user ratings
- numVotes - number of votes the title has received

In [25]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360911 entries, 0 to 1360910
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1360911 non-null  object 
 1   averageRating  1360911 non-null  float64
 2   numVotes       1360911 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 31.1+ MB


In [26]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1894
3,tt0000004,5.5,178
4,tt0000005,6.2,2677


In [27]:
print(f'There are {ratings.shape[0]} rows and {ratings.shape[1]} columns in the title_basics dataset.')

There are 1360911 rows and 3 columns in the title_basics dataset.


In [28]:
## Replace "\N" with np.nan
ratings.replace({'\\N':np.nan}, inplace=True)

In [29]:
## Keep only US movies
us_movies = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[us_movies]

In [30]:
## Cleaned and filtered data
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 512157 entries, 0 to 1360887
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         512157 non-null  object 
 1   averageRating  512157 non-null  float64
 2   numVotes       512157 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.6+ MB


## Save the data

In [31]:
## Create a Data folder
os.makedirs('Data/', exist_ok=True) 

In [32]:
## Save current dataframes to files
akas.to_csv("Data/title_akas.csv.gz", compression='gzip', index=False)
basics.to_csv("Data/title_basics.csv.gz", compression='gzip', index=False)
ratings.to_csv("Data/title_ratings.csv.gz", compression='gzip', index=False)

## Preview saved data

In [36]:
akas_clean = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas_clean.info()
print(f'There are {akas_clean.shape[0]} rows and {akas_clean.shape[1]} columns in the clean and filtered title_akas dataset.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472694 entries, 0 to 1472693
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1472694 non-null  object
 1   ordering         1472694 non-null  int64 
 2   title            1472694 non-null  object
 3   region           1472694 non-null  object
 4   language         4171 non-null     object
 5   types            984741 non-null   object
 6   attributes       47636 non-null    object
 7   isOriginalTitle  1472694 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 80.1+ MB
There are 1472694 rows and 8 columns in the clean and filtered title_akas dataset.


In [37]:
basics_clean = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics_clean.info()
print(f'There are {basics_clean.shape[0]} rows and {basics_clean.shape[1]} columns in the clean and filtered title_basics dataset.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82146 entries, 0 to 82145
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82146 non-null  object 
 1   titleType       82146 non-null  object 
 2   primaryTitle    82146 non-null  object 
 3   originalTitle   82146 non-null  object 
 4   isAdult         82146 non-null  bool   
 5   startYear       82146 non-null  object 
 6   endYear         0 non-null      float64
 7   runtimeMinutes  82146 non-null  float64
 8   genres          82146 non-null  object 
dtypes: bool(1), float64(2), object(6)
memory usage: 5.1+ MB
There are 82146 rows and 9 columns in the clean and filtered title_basics dataset.


In [38]:
ratings_clean = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings_clean.info()
print(f'There are {ratings_clean.shape[0]} rows and {ratings_clean.shape[1]} columns in the clean and filtered title_ratings dataset.')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512157 entries, 0 to 512156
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         512157 non-null  object 
 1   averageRating  512157 non-null  float64
 2   numVotes       512157 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.7+ MB
There are 512157 rows and 3 columns in the clean and filtered title_ratings dataset.
